In [1]:
import argparse
import os
import os.path as osp
from tqdm import tqdm

import shutil
import time
import warnings
warnings.filterwarnings('ignore')

import mmseg_custom
import mmcv_custom

import mmcv
import torch
from mmcv import Config
from mmcv.cnn.utils import revert_sync_batchnorm
from mmcv.parallel import MMDataParallel, MMDistributedDataParallel
from mmcv.runner import (get_dist_info, init_dist, load_checkpoint,
                         wrap_fp16_model)
from mmcv.utils import DictAction

from mmseg import digit_version
from mmseg.apis import multi_gpu_test, single_gpu_test
from mmseg.datasets import build_dataloader, build_dataset
from mmseg.models import build_segmentor
# from mmseg.utils import setup_multi_processes

In [2]:
# config 파일과 work_dir, epoch(model)을 설정해주세요.
cfg = Config.fromfile('./work_dirs/mask2former_beit_adapter_base_512_40k_upstage_ss_PL/mask2former_beit_adapter_base_512_40k_upstage_ss_PL.py')
cfg.work_dir = './work_dirs/mask2former_beit_adapter_base_512_40k_upstage_ss_PL'
epoch = 'best_mIoU_iter_8000'


cfg.seed=2022
cfg.gpu_ids = [1]
cfg.model.train_cfg = None
cfg.data.test.test_mode = True

# build dataset & dataloader
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=1,
        workers_per_gpu=4,
        dist=False,
        shuffle=False)

# checkpoint path
checkpoint_path = os.path.join(cfg.work_dir, f'{epoch}.pth')

model = build_segmentor(cfg.model, test_cfg=cfg.get('test_cfg')) # build detector
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu') # ckpt load

model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

2023-01-05 12:32:20,595 - mmseg - INFO - Loaded 819 images
2023-01-05 12:32:22,458 - mmseg - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc_norm.weight, fc_norm.bias, head.weight, head.bias

missing keys in source state_dict: blocks.0.attn.relative_position_index, blocks.1.attn.relative_position_index, blocks.2.attn.relative_position_index, blocks.3.attn.relative_position_index, blocks.4.attn.relative_position_index, blocks.5.attn.relative_position_index, blocks.6.attn.relative_position_index, blocks.7.attn.relative_position_index, blocks.8.attn.relative_position_index, blocks.9.attn.relative_position_index, blocks.10.attn.relative_position_index, blocks.11.attn.relative_position_index



Position interpolate for blocks.0.attn.relative_position_bias_table from 27x27 to 63x63
x = [-31.000181806480445, -26.433792368127026, -22.410250408812363, -18.86502280242512, -15.741248051675145, -12.988824293086934, -10.563605718981357, -8.426694527950382, -6.5438170474999815, -4.884774022564157, -3.422956253130848, -2.134917812347412, -1, 0, 1, 2.134917812347412, 3.422956253130848, 4.884774022564157, 6.5438170474999815, 8.426694527950382, 10.563605718981357, 12.988824293086934, 15.741248051675145, 18.86502280242512, 22.410250408812363, 26.433792368127026, 31.000181806480445]
dx = [-31. -30. -29. -28. -27. -26. -25. -24. -23. -22. -21. -20. -19. -18.
 -17. -16. -15. -14. -13. -12. -11. -10.  -9.  -8.  -7.  -6.  -5.  -4.
  -3.  -2.  -1.   0.   1.   2.   3.   4.   5.   6.   7.   8.   9.  10.
  11.  12.  13.  14.  15.  16.  17.  18.  19.  20.  21.  22.  23.  24.
  25.  26.  27.  28.  29.  30.  31.]
Position interpolate for blocks.1.attn.relative_position_bias_table from 27x27 to 63x63
x

In [3]:
output = single_gpu_test(model, data_loader) # output 계산

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 819/819, 7.7 task/s, elapsed: 107s, ETA:     0s

In [4]:
import json
import pandas as pd

# 저장 위치를 설정하세요.
save_dir = "/opt/ml/input/code/submission"
csv_name = 'mask2former_beit_adapter_base_PL'

with open("/opt/ml/input/data/test.json", "r") as f:
    test = json.load(f)

# sample_submisson.csv 열기
submission = pd.read_csv('/opt/ml/input/code/submission/sample_submission.csv', index_col=None)

# out[::2,::2]
# for id, out in zip(test['images'], output):
#     submission = submission.append({"image_id" : id['file_name'], "PredictionString" : " ".join(map(str, out[::2,::2].flatten().tolist()))}, 
#                                    ignore_index=True)


for id, out in tqdm(zip(test['images'], output)):
    # maxpooling
    input_size = 512
    output_size = 256
    bin_size = input_size // output_size

    out = out.reshape((1,512,512))
    result = out.reshape((1, output_size, bin_size, 
                            output_size, bin_size)).max(4).max(2)

    submission = submission.append({"image_id" : id['file_name'], "PredictionString" : " ".join(map(str, result.flatten().tolist()))}, 
                                   ignore_index=True)

# submission.csv로 저장
submission.to_csv(osp.join(save_dir,f'{csv_name}.csv'), index=False)

819it [00:12, 63.79it/s]
